# app

> This module contains code for the running of the application

In [32]:
#| default_exp app

In [33]:
#| hide
from nbdev.showdoc import *

These are the relevant library calls to build out the UI and the calls to the model in order to set up the manuscript selection buttons.

In [34]:
#| export
from dash import Dash, State, Input, Output, callback, dcc, html
import dash_bootstrap_components as dbc
from glyptodon.annotation import *
from glyptodon.classes import *
from glyptodon.export import *
from glyptodon.information import *
from glyptodon.manuscriptFiles import *
from glyptodon.selection import *
import re
import os

## Bootstrap Object Calls|

In [35]:
#| export
selectionKey, selectionLayout = createSelectionLayout()
print(selectionKey)

centuries, informationLayout = createInformationLayout()

annotationLayout = createAnnotationLayout()

exportLayout = createExportLayout()

{'EPARCHOS': ('/home/dc/glyptodon/nbs/manuscripts/eparchos', {'Work': 'EPARCHOS', 'Author': 'None', 'Language': 'None', 'Country': 'None', 'City': 'None', 'Institution': 'None', 'Centuries': '1st to 20th Centuries'}), 'Stavronikita Monastery Greek handwritten document Collection no.53': ('/home/dc/glyptodon/nbs/manuscripts/stvrnktmnstrygrkcllctnn.53', {'Work': 'Stavronikita Monastery Greek handwritten document Collection no.53', 'Author': 'Anonymous', 'Language': 'Greek', 'Country': 'Greece', 'City': 'Mount Athos', 'Institution': 'Stavronikita Monastery', 'Centuries': '14th Century'})}


## Bootstrap Application

This makes the bootstrap version of the application. All ids are kept in [kebab-case](https://www.freecodecamp.org/news/snake-case-vs-camel-case-vs-pascal-case-vs-kebab-case-whats-the-difference/).

In [36]:
#| export
app = Dash(
    external_stylesheets=[dbc.themes.BOOTSTRAP]
)

app.layout = html.Div(
    [
        dcc.Tabs(
            id="tabs-object",
            value="selection",
            children=[
                dcc.Tab(
                    label="Selection",
                    value="selection",
                    children=[
                        selectionLayout
                    ],
                ),
                dcc.Tab(
                    label="Information",
                    value="information",
                    children=[
                        informationLayout
                    ],
                ),
                dcc.Tab(
                    label="Annotation",
                    value="annotation",
                    children=[
                        annotationLayout
                    ],
                ),
                dcc.Tab(
                    label="Export",
                    value="export",
                    children=[
                        exportLayout
                    ],
                ),
            ],
        ),
#        html.Div(id="current-tab"),
        html.Div(
            children="no callback",
            id="dummy-output",
            style={"display": "none"},
        ),
    ]
)

## A callback that effects layout

The ```manuscriptSelect``` object has a **Create New Manuscript** option. If the user selects that option, we want a different layout to be available for the ```selection``` tab. This code cell sets up a [callback](https://dash.plotly.com/basic-callbacks) for the object to make sure that there is a ```bool``` to handle the conditional logic. Additionally, this callback is used to make sure that the correct manuscript is selected from the available options.

The output is a 'dummy' object created in order to have a function with no real output as per the [suggestions here](https://community.plotly.com/t/app-callback-without-an-output/5502/36).

In [37]:
#| export
newManuscript = False
selectedManuscript = selectionKey[
    "Stavronikita Monastery Greek handwritten document Collection no.53"
]

@callback(
    Output("work", "placeholder",),
    Output("author", "placeholder",),
    Output("language", "placeholder",),
    Output("country", "placeholder",),
    Output("city", "placeholder",),
    Output("institution", "placeholder",),
    Output("centuries-slider", "value",),
    Output("uploader-container","style"), # This determines if uploaders are displayed
    Input("manuscript-select", "value"),
    suppress_callback_exceptions=True,
)
def selectManuscript(work):
    global selectedManuscript
    global selectionKey
    if work == "Create New Manuscript":
        newManuscript = True
        selectedManuscript = None
        return "", "", "", "", "", "", [1, 20], {"display": "block"}
    else:
        selectedManuscript = selectionKey[work]
        work = selectedManuscript[1]["Work"]
        author = selectedManuscript[1]["Author"]
        language = selectedManuscript[1]["Language"]
        country = selectedManuscript[1]["Country"]
        city = selectedManuscript[1]["City"]
        institution = selectedManuscript[1]["Institution"]

        ### Converts the string containing centuries into list containing the centuries as integers
        # This stores the string as a list of words. There are strings with two words and four words
        centuriesAsList = selectedManuscript[1]["Centuries"].split()

        # This picks out the relevant words and strips them of everything but the integer values
        if len(centuriesAsList) == 2:
            centuriesValue = [
                int(re.sub("[^0-9]", "", centuriesAsList[0])),
                int(re.sub("[^0-9]", "", centuriesAsList[0])),
            ]
        else:
            centuriesValue = [
                int(re.sub("[^0-9]", "", centuriesAsList[0])),
                int(re.sub("[^0-9]", "", centuriesAsList[2])),
            ]
        return work, author, language, country, city, institution, centuriesValue, {"display": "none"}

## Finalize selection callback

When the ```finalizeSelection``` button is clicked, we need to start creating the figures that will be used in the ```annotation-figure``` object. This is a CPU intensive process (taking roughly 20 seconds to perform on my machine), so it isn't something performed trivially.

In [38]:
#| export
@callback(
    Output("tabs-object","value", allow_duplicate=True),
    Input("finalize-selection", "n_clicks"),
    prevent_initial_call=True,
)
def finalizeSelectionCallback(clicks):
    return "information"

## Save and Continue callback

When the ```save-and-continue``` button is clicked, we need to be able to save data to a current ```.cfg``` file in the manuscript directory as well as create a new manuscript directory. This is dependent on the state of ```manuscript-select```. A new manuscript will require a new directory from ```createManuscriptDirectory``` to set up the directory initially as well as the usage of ```saveImage``` [and eventually ```saveTranscript```] to put the relevant files into the directory. Additionally, to make sure that the rest of the program still works, the ```selectedManuscript``` value (which is a global variable) will have to be updated to the value returned from ```createManuscriptDirectory```

In [39]:
#| export
@callback(
    Output("tabs-object", "value", allow_duplicate=True),
    Output("manuscript-select", "value"),
    Output("manuscript-select", "options"),
    Output("page-selector", "options"),
    Output("page-selector", "value"),
    Input("save-and-continue", "n_clicks"),
    # Input objects
    State("work", "value"),
    State("author", "value"),
    State("language", "value"),
    State("country", "value"),
    State("city", "value"),
    State("institution", "value"),
    State("centuries-slider", "value"),
    # Upload objects
    State("upload-images", "contents"),
    State("upload-images", "filename"),
    State("upload-manuscripts","contents"),
    State("upload-manuscripts","filename"),
    # Conditional object
    State("manuscript-select", "value"),
    State("manuscript-select", "options"),
    prevent_initial_call=True,
)
def saveNContinuteCallback(
    clicks, # Input save-and-continue
    work, # State work
    author, # State author
    language, # State language
    country, # State country
    city, # State city
    institution, # State institution
    centuriesValue, # State centuries-slider
    imContents, # State upload-images
    imFilenames, # State upload-images
    manContents, # State upload-manuscripts
    manFilenames, # State upload-manuscripts
    manSelectVal, # State manuscript-select value
    manSelectOpts, # State manuscript select options
):
    global selectedManuscript
    global centuries
    global selectionKey
    centuriesData = ""
    if centuriesValue[0] == centuriesValue[1]:
        centuriesData = centuries[centuriesValue[0]] + " Century"
    else:
        centuriesData = (
            centuries[centuriesValue[0]]
            + " to "
            + centuries[centuriesValue[1]]
            + " Centuries"
        )
    
    information = {
        "Work": work,
        "Author": author,
        "Language": language,
        "Country": country,
        "City": city,
        "Institution": institution,
        "Centuries": centuriesData,
    }

    if manSelectVal == "Create New Manuscript":
        selectedManuscript = (createManuscriptDirectory(information), information)
        saveImages(imContents, imFilenames, selectedManuscript[0])
        saveTranscripts(manContents, manFilenames, selectedManuscript[0])
        
        manuscripts = currentManuscripts()
        selectionKey = {}
        selectionNames = []
        for manuscript in manuscripts:
            selectionNames.append(manuscript[1]["Work"])
            selectionKey[selectionNames[-1]] = manuscript
        
        manSelectVal = information["Work"]
        manSelectOpts = selectionNames + ["Create New Manuscript"]
        
        selectedManuscript = selectionKey[information["Work"]]
        
        dropdownOptions = []
        relativePaths = manuscriptImages(selectedManuscript[0])

        index = 1
        for path in relativePaths:
            imageName = path.split("/")[-1]
            if imageName[0] != ".":
                dropdownOptions.append({"label": f"Page {index}", "value": path})
                index = index + 1
        
        return "annotation", manSelectVal, manSelectOpts, dropdownOptions, dropdownOptions[0]["value"]
    else:
        updateMetadata(selectedManuscript[0], information)
        
        dropdownOptions = []
        relativePaths = manuscriptImages(selectedManuscript[0])

        index = 1
        for path in relativePaths:
            imageName = path.split("/")[-1]
            if imageName[0] != ".":
                dropdownOptions.append({"label": f"Page {index}", "value": path})
                index = index + 1
        
        return "annotation", manSelectVal, manSelectOpts, dropdownOptions, dropdownOptions[0]["value"]

## Page Selector callback

When the page selector changes its value, we want to make it so that a particular image is displayed as the figure in the ```Graph``` object with ```annotation-figure``` id.

In [40]:
#| export
@callback(
    Output("annotation-figure", "figure"),
    Input("page-selector", "value"),
    prevent_initial_call=True,
)
def pageSelectorCallback(path):
    global selectedManuscript
    fig = createAnnotationFigure(path)

    imageName = path.split("/")[-1]  # This takes the file name in the directory
    imageName = imageName.split(".")[0]  # This assumes

    statesDirectory = os.path.join(selectedManuscript[0], "states")
    linesDirectory = os.path.join(statesDirectory, "lines")
    bboxesDirectory = os.path.join(statesDirectory, "bboxes")
    
    # Instantiate lines from csv and add them to figure
    for file in os.listdir(linesDirectory):
        fileName = file.split(".")[0]
        if fileName == imageName:
            figLines = Line.csvToLines(os.path.join(linesDirectory, file))

            for line in figLines:
                fig.add_shape(
                    editable = True,
                    type=line.type,
                    x0=line.x0,
                    y0=line.y0,
                    x1=line.x1,
                    y1=line.y1,
                    line=line.line,
                    opacity=line.opacity,
                )

    # Instantiate bboxes from csv and add them to figure
    for file in os.listdir(bboxesDirectory):
        fileName = file.split(".")[0]
        if fileName == imageName:
            figBBoxes = BBox.csvToBBoxes(os.path.join(bboxesDirectory, file))
            
            for bbox in figBBoxes:
                fig.add_shape(
                    editable = True,
                    type=bbox.type,
                    x0=bbox.x0,
                    y0=bbox.y0,
                    x1=bbox.x1,
                    y1=bbox.y1,
                    line=bbox.line,
                    opacity=bbox.opacity,
                )

    return fig

## Save Shapes callback

This callback takes the current [```State```](https://dash.plotly.com/basic-callbacks#dash-app-with-state) of the figure and the page selector to convert that input into sorted ```BBox``` and ```Line``` objects that are then saved to ```csv``` files. There are several components to this code, so they will be explained here with markdown rather than commenting through it:

### callback
```python
@callback(
    Output("dummy-output","children"),
    Input("save-shapes", "n_clicks"),
    State("annotation-figure", "relayoutData"),
    State("page-selector", "value"),
    prevent_initial_call=True,
    allow_duplicate=True,
)
```
This callback does not actually do anything to update values inside the program. Rather, it takes values and saves them. However, all ```Dash``` callbacks have to have a specified ```Output```. Because of that, a dummy output has been made with an ```html.Div``` object. Additionally, to avoid continuously updating storage this method is only called when the ```save-shapes``` button is clicked. The ```State``` elements are only called when the ```Input``` is triggered. Next, the initial call is prevented as objects referenced in this callback will not have well defined traits until other events have happened. Lastly, the allow duplicate is in place to allow the dummy output to be used in another callback.

### function definition
```python
def saveShapesCallback(clicks, shapes, path):
```
This method call has to take positional arguments based on the order of the callback. So, the ```save-shapes``` variable of ```n_clicks``` corresponds to ```clicks``` in the positional arguments. The ```shapes``` variable gives us the state of shapes on the figure, allowing us to dig through a list of dictionaries (oddly enough passed in as a dictionary where the list is keyed to ```"shapes"```) to extract the shape data. The ```path``` will tell us which image we are using.

### data extraction
```python
dictLines = []
dictBBoxes = []
for shape in shapes["shapes"]:
    if shape["type"] == "line":
        dictLines.append(shape)
    if shape["type"] == "rect":
        dictBBoxes.append(shape)
```
These first set of code just take the ```shapes``` argument and sorts the dictionaries inside the list into either the line or bbox category.

### Line construction
```python
lines = []
for line in dictLines:
    bboxes.append([])
    lines.append(
        Line(
            x0=line["x0"],
            y0=line["y0"],
            x1=line["x1"],
            y1=line["y1"],
        )
    )
Line.sortLines(lines)
```
This takes the extracted line dictionary and uses the data stored inside to instantiate ```Line``` objects for every dictionary. It then calls the ```sortLines``` method to not only sort the ```lines``` list but also assign ```index``` values to each line.

### BBox construction
```python
tempBBoxes = []
for bbox in dictBBoxes:
    tempBBoxes.append(
        BBox(
            x0=int(bbox["x0"]),
            y0=int(bbox["y0"]),
            x1=int(bbox["x1"]),
            y1=int(bbox["y1"]),
        )
    )
```
This takes the extracted bbox dictionary and uses the data stored inside to instantiate ```BBox``` objects for every dictionary. Unfortunately, we want to sort the ```BBox``` objects along two possible coordinates. We want to get bboxes sorted into which text line they are a part of and we want them to be sorted along that line as well. This suggests a coordinate system for the bboxes where any bbox can be assigned to a coordinate $(L,I)$ where $L$ is for the line (a correspondence of the ```Line``` objects ```index``` trait and the ```BBox``` objects ```lineNo``` trait) and  $I$ is for the index of the box on that line (meaning that the further left a word/bbox is on a line the lesser the index). However, that sorting has to happen after construction.

### BBox lineNo sorting
```python
bboxes = []
for line in lines:
    bboxes.append([])
    for bbox in tempBBoxes:
        if bbox.isLine(line):
            bboxes[-1].append(bbox)
```
This takes the bboxes stored temporarily and stores them in lists inside of the ```bboxes``` list. First, it constructs an empty list. It then uses the ```isLine``` function to determine if a given ```BBox``` can be assigned to some $L$ (```lineNo```) for the current ```Line``` object in the for loop. Because the ```lines``` list has been sorted, we can be confident that the list we are assigning a ```BBox``` to at the tail end of the ```bboxes``` list will be at an index in ```bboxes``` of ```line.index - 1```.

### BBox index sorting and storage preparation
```python
flattenedBBoxes = []
for line in bboxes:
    BBox.sortBBoxes(line)
    flattenedBBoxes = flattenedBBoxes + line
```
This takes the two dimensional ```bboxes``` list, sorts each list inside with ```sortBBoxes```, and then puts it into one array for easy saving purposes. The ```sortBBoxes``` method is used to assign an ```index``` value for each ```BBox```, necessary for reconstructing the ```BBox``` objects or exporting the generated ```csv``` in another format. The ```flattenedBBoxes``` converts the 2-dimensional ```bboxes``` into a one dimensional list to pass into the ```BBox``` csv constructor method.

### Saving objects
```python
imageName = path.split("/")[-1]
imageName = imageName.split(".")[0]

statesDirectory = os.path.join(selectedManuscript[0], "states")
linesDirectory = os.path.join(statesDirectory, "lines")
bboxesDirectory = os.path.join(statesDirectory, "bboxes")

Line.linesToCSV(linesDirectory, lines, imageName)
BBox.bboxesToCSV(bboxesDirectory, flattenedBBoxes, imageName)
```
This uses the ```path``` argument to extract a filename as ```imageName``` (reusing code from earlier). Then, the pathway to the ```bboxes``` and ```lines``` directories are established and the particular files written to with the ```...ToCSV``` methods.

### Dummy output
```python
dummy = ["1","2","3"]
return dummy
```
This is in the code because of the ```Dash``` requirement that any callback function has a distinct output relating to one of the components of the application. This method does not actually have any reason to pass output to any object in the application. Thus, the dummy output is used.

In [41]:
#| export
@callback(
    Output("dummy-output","children", allow_duplicate=True),
    Input("save-shapes", "n_clicks"),
    State("annotation-figure", "relayoutData"),
    State("page-selector", "value"),
    prevent_initial_call=True,
)
def saveShapesCallback(clicks, shapes, path):
    global selectedManuscript
    dictLines = []
    dictBBoxes = []
    for shape in shapes["shapes"]:
        if shape["type"] == "line":
            dictLines.append(shape)
        if shape["type"] == "rect":
            dictBBoxes.append(shape)
    
    lines = []
    for line in dictLines:
        lines.append(
            Line(
                x0=line["x0"],
                y0=line["y0"],
                x1=line["x1"],
                y1=line["y1"],
            )
        )
    Line.sortLines(lines)
    
    tempBBoxes = []
    for bbox in dictBBoxes:
        tempBBoxes.append(
            BBox(
                x0=int(bbox["x0"]),
                y0=int(bbox["y0"]),
                x1=int(bbox["x1"]),
                y1=int(bbox["y1"]),
            )
        )

    bboxes = []
    for line in lines:
        bboxes.append([])
        for bbox in tempBBoxes:
            if bbox.isLine(line):
                bboxes[-1].append(bbox)

    flattenedBBoxes = []
    for line in bboxes:
        BBox.sortBBoxes(line)
        flattenedBBoxes = flattenedBBoxes + line

    imageName = path.split("/")[-1]
    imageName = imageName.split(".")[0]

    statesDirectory = os.path.join(selectedManuscript[0], "states")
    linesDirectory = os.path.join(statesDirectory, "lines")
    bboxesDirectory = os.path.join(statesDirectory, "bboxes")

    Line.linesToCSV(linesDirectory, lines, imageName)
    BBox.bboxesToCSV(bboxesDirectory, flattenedBBoxes, imageName)
    
    dummy = ["1","2","3"]
    return dummy

## Line Number callback

This callback is used to create line numbers in the textarea for ease of transcription. Unfortunately, this is not something that can be done through Python. The only [guide](https://webtips.dev/add-line-numbers-to-html-textarea) I could find requires HTML objects not on offer in ```Dash```, specifically the ```Array```. For that reason, rather than adding line numbers to the text area independent of its text value, the value of the text area will be updated to include line numbers.

In [42]:
#| export
@callback(
    Output("annotation-text-area","value"),
    Input("annotation-figure", "relayoutData"),
    State("annotation-text-area","value"),
    prevent_initial_call=True,
)
def lineNumberCallback(shapes, currentText):
    numLines = 0
    for shape in shapes["shapes"]:
        if shape["type"] == "line":
            numLines = numLines + 1
    
    currentLines = currentText.split("\n")
    newLines = []
    for i in range(1,numLines+1):
        if i > len(currentLines):
            if i < 10:
                newLines.append(f"0{i}:\n")
            else:
                newLines.append(f"{i}:\n")
        else:
            if currentLines[i-1][0:2].isnumeric() and int(currentLines[i-1][0:2]) == i:
                newLines.append(currentLines[i-1] + "\n")
            elif i < 10:
                newLines.append(f"0{i}:\n")
            else:
                newLines.append(f"{i}:\n")
    
    newValue = ""
    for line in newLines:
        newValue = newValue + line
    
    return newValue

## Save Annotation callback

Like the callback for the ```save-shapes``` button, the callback for the ```save-annotation``` button saves the current shapes from the figure. However, instead of saving without an annotation, it saves with an ```annotation``` property assigned to each ```BBox``` and a ```text``` property assigned to each ```Line```.

In [43]:
#| export
@callback(
    Output("dummy-output", "children", allow_duplicate=True),
    Input("save-annotation", "n_clicks"),
    State("annotation-figure", "relayoutData"),
    State("page-selector", "value"),
    State("annotation-text-area", "value"),
    prevent_initial_call=True,
)
def saveAnnotationCallback(clicks, shapes, path, currentText):
    global selectedManuscript
    dictLines = []
    dictBBoxes = []
    for shape in shapes["shapes"]:
        if shape["type"] == "line":
            dictLines.append(shape)
        if shape["type"] == "rect":
            dictBBoxes.append(shape)

    currentLines = currentText.split("\n")
    currentWords = []
    for line in currentLines:
        currentWords.append(line[4:].split(" "))

    lines = []
    for line in dictLines:
        lines.append(
            Line(
                x0=line["x0"],
                y0=line["y0"],
                x1=line["x1"],
                y1=line["y1"],
            )
        )
    Line.sortLines(lines)

    for line in lines:
        line.text = currentLines[line.index - 1][4:]

    tempBBoxes = []
    for bbox in dictBBoxes:
        tempBBoxes.append(
            BBox(
                x0=int(bbox["x0"]),
                y0=int(bbox["y0"]),
                x1=int(bbox["x1"]),
                y1=int(bbox["y1"]),
            )
        )

    bboxes = []
    for line in lines:
        bboxes.append([])
        for bbox in tempBBoxes:
            if bbox.isLine(line):
                bboxes[-1].append(bbox)

    flattenedBBoxes = []
    for line in bboxes:
        BBox.sortBBoxes(line)
        flattenedBBoxes = flattenedBBoxes + line

    for line in bboxes:
        if len(line) == 0:
            pass
        elif len(line) == len(currentWords[line[0].lineNo - 1]):
            for bbox in line:
                bbox.annotation = currentWords[bbox.lineNo - 1][bbox.index - 1]

    imageName = path.split("/")[-1]
    imageName = imageName.split(".")[0]

    statesDirectory = os.path.join(selectedManuscript[0], "states")
    linesDirectory = os.path.join(statesDirectory, "lines")
    bboxesDirectory = os.path.join(statesDirectory, "bboxes")

    Line.linesToCSV(linesDirectory, lines, imageName)
    BBox.bboxesToCSV(bboxesDirectory, flattenedBBoxes, imageName)

    dummy = ["1", "2", "3"]
    return dummy

## Next Tab callback

This is a simple callback for the ```next-tab``` button in the ```annotation``` tab. It moves the tab from ```annotation``` to ```export```.

In [44]:
#| export
@callback(
    Output("tabs-object", "value", allow_duplicate=True),
    Input("next-tab", "n_clicks"),
    prevent_initial_call=True,
)
def nextTabCallback(clicks):
    return "export"

## Export Manuscript callback

This callback uses the ```export-manuscript``` button to call the ```zipManuscript()``` method and then use the ```Download``` object to automatically download the zipped manuscript.

In [45]:
#| export
@callback(
    Output("export-download", "data"),
    Input("export-button", "n_clicks"),
    State("export-name", "value"),
    State("directory-options", "value"),
    prevent_initial_call=True,
)
def exportManuscriptCallback(clicks, name, options):
    global selectedManuscript
    path = zipManuscript(options, selectedManuscript[0], name)
    return dcc.send_file(path)

## Running the application

This cell contains the code that runs the application. It comes logically after all the prior code. To see the application on the local server, use http://127.0.0.1:8050/ as the website link to your locally run server.

In [46]:
#| export
if __name__ == "__main__":
    app.run(debug=True)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
KeyError: 'shapes'



In [16]:
#| hide
import nbdev

nbdev.nbdev_export()